this notebook train doc2vec model on only events captions

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import tools
import d2vlib
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# init paths to data and models
DATA_PATH = '/data/'

MODEL_NAME = 'd2v_events'

MODEL_PATH = DATA_PATH + f'models/{MODEL_NAME}/'
EVENTS_PATH = DATA_PATH + 'events/lem/events.csv'


TMP_PATH = f'{DATA_PATH}tmp/{MODEL_NAME}/'
    
    
n_clusters = list(range(5, 151)) # the number of clusters to choose the optimal number

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

loading events and preparing them for training

In [6]:
df = tools.read_events(EVENTS_PATH)
documents = []

for index, row in df.iterrows():
    documents.append(TaggedDocument(row.description.split(), [index]))
    
print(f'loaded all events: {len(documents)}')

loaded all events: 10214


training doc2vec model

In [7]:
model = Doc2Vec(documents, vector_size=300, window=10, min_count=5, negative=10, dm=1, dbow_words=1, epochs=200, workers=35)
!mkdir {MODEL_PATH}
model.save(MODEL_PATH + 'mdl')

mkdir: cannot create directory ‘/data/models/d2v_events/’: File exists


This cell prepare events vectors:
1. infering vectors for events by the doc2vec model (vectors for euclidean distance)
2. building normilized vectors for cosine distance
3. building 2d embedding by t-sne for euclidean distance and cosine distance
4. saving data to tmp dir

In [ ]:
df = tools.read_events(EVENTS_PATH)
X = tools.build_vectors_doc2vec(df, MODEL_PATH)
df = tools.calculate_2d(df, X)

!mkdir {TMP_PATH}
np.save(TMP_PATH + 'X.npy', X)
df.to_csv(r'' + TMP_PATH + 'df.csv', index=False)

clustering of events, for each n from n_samples labels is saved in dataframe <br>
evaluating f1-score for each clustering

In [ ]:
df = tools.read_events(TMP_PATH + 'df.csv')
df_cross = pd.read_csv('cross_valid_union.csv')
X = np.load(TMP_PATH + 'X.npy')

# def k_means_list(n_clusters: list, df: pd.DataFrame, X: np.array, n_init=50):
df = tools.k_means_list(n_clusters, df, X)
# def calc_scores_list(n_clusters:list, df: pd.DataFrame, pairs_df: pd.DataFrame):
df_scores = tools.calc_scores_list(n_clusters, df, df_cross)

df.to_csv(r'' + TMP_PATH + 'df_km.csv', index=False)
df_scores.to_csv(r'' + TMP_PATH + 'scores_km.csv', index=False)

In [13]:
df = tools.read_events(TMP_PATH + 'df.csv')
df_cross = pd.read_csv('cross_valid_union.csv')
X = np.load(TMP_PATH + 'X.npy')

# def k_means_list(n_clusters: list, df: pd.DataFrame, X: np.array, n_init=50):
df = tools.agglomerative_list(n_clusters, df, X)
# def calc_scores_list(n_clusters:list, df: pd.DataFrame, pairs_df: pd.DataFrame):
df_scores = tools.calc_scores_list(n_clusters, df, df_cross)

df.to_csv(r'' + TMP_PATH + 'df_ag.csv', index=False)
df_scores.to_csv(r'' + TMP_PATH + 'scores_ag.csv', index=False)

finding the best f1-score and plot f1-score for diffferent number of clusters

In [19]:
reload(tools)
df = tools.read_events(TMP_PATH + 'df_km.csv')
scores = pd.read_csv(TMP_PATH + 'scores_km.csv')
# def find_best_score(scores_df: pd.DataFrame, score_name: str):
best = tools.find_best_score(scores, 'f1')
print(best)
# def plot_score(scores_df: pd.DataFrame, y, x='n_clusters', x_title='n clusters', y_title='score value'):
tools.plot_score(scores, 'f1').show()
# def plot_events(df: pd.DataFrame, color:str or int, hover_name='title'):
tools.plot_events(df, best['n_clusters'])
# def calc_centroids(df: pd.DataFrame, n:int, hashtags_size=15):
centroids = tools.calc_centroids(df, best['n_clusters'])
# def plot_centroids(df_centroids, size_max=100, size_text_tags=0, min_size=0, max_size=None):
fig, _ = tools.plot_centroids(centroids, size_max=80, size_text_tags=1)
fig.show()

{'name': 'f1', 'value': 0.7134767836919593, 'n_clusters': 29}


In [20]:
reload(tools)
df = tools.read_events(TMP_PATH + 'df_ag.csv')
scores = pd.read_csv(TMP_PATH + 'scores_ag.csv')
# def find_best_score(scores_df: pd.DataFrame, score_name: str):
best = tools.find_best_score(scores, 'f1')
print(best)
# def plot_score(scores_df: pd.DataFrame, y, x='n_clusters', x_title='n clusters', y_title='score value'):
tools.plot_score(scores, 'f1').show()
# def plot_events(df: pd.DataFrame, color:str or int, hover_name='title'):
tools.plot_events(df, best['n_clusters'])
# def calc_centroids(df: pd.DataFrame, n:int, hashtags_size=15):
centroids = tools.calc_centroids(df, best['n_clusters'])
# def plot_centroids(df_centroids, size_max=100, size_text_tags=0, min_size=0, max_size=None):
fig, _ = tools.plot_centroids(centroids, size_max=80, size_text_tags=1)
fig.show()

{'name': 'f1', 'value': 0.4530082579630358, 'n_clusters': 12}
